## Index
[Importing modules](#Importing-required-modules)\
[Importing data](#Importing-file-and-unzipping)\
[Cleaning](#Cleaning)

### Importing required modules

* `requests` used for importing the downloaded zip file
* `tqdm` for progress bars, file size, elapsed time and download speeds
* `zipfile` for opening the zip file
* `concurrent.futures` for multiprocessing
* `pandas` as datatype
* `numpy` for number formatting

In [94]:
import requests
from tqdm.notebook import tqdm
import zipfile
import concurrent.futures
import pandas as pd
import numpy as np
from io import StringIO

### Importing file and unzipping

In [81]:
#Download data to hospitalPriceData.zip in working directory
hospitalPriceData_zip_url = 'https://www.dolthub.com/csv/dolthub/hospital-price-transparency-v3/iptu80riko4il5qij5asr8nalodeo9ut?include_bom=0'

#session to efficiently use data streaming
session = requests.Session()
response = session.get(hospitalPriceData_zip_url, stream= True)
total_size_in_bytes = int(response.headers.get('content-length', 0))
block_size = 1024**2 #1 MiB blockst
progress_bar = tqdm(total= total_size_in_bytes, unit= 'B', unit_scale= True)
with open("hospitalPriceData.zip", "wb") as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()

#2.31 GB, TOOK 1hr 15min WITH 675mb ETHERNET (though dolthub was the bottleneck)

KeyboardInterrupt: 

In [ ]:
#%%time

#Extract files from hospitalPriceData.zip
zf = zipfile.ZipFile('hospitalPriceData.zip')
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(zf.extractall(), zf.infolist())

#3min 18s wall time normal
#3min 7s wall time multithreaded

CPU times: user 1min 19s, sys: 35.9 s, total: 1min 55s
Wall time: 3min 7s


### Cleaning

In [ ]:
hospitals_df = pd.read_csv('hospitals.csv', low_memory=False)

In [ ]:
%%time
prices_df = pd.read_csv(stringIO('prices.csv'), low_memory=True)
#11 min low_memory= True

<decorator-gen-61>:2: DtypeWarning: Columns (2,3,4,5,8) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 4min 34s, sys: 3min 56s, total: 8min 30s
Wall time: 11min 58s


In [ ]:
#print(hospitals_df.head())
print(hospitals_df.head())

  cms_certification_num                                   name  \
0                010001        SOUTHEAST HEALTH MEDICAL CENTER   
1                010005  MARSHALL MEDICAL CENTERS SOUTH CAMPUS   
2                010006           NORTH ALABAMA MEDICAL CENTER   
3                010007               MIZELL MEMORIAL HOSPITAL   
4                010008            CRENSHAW COMMUNITY HOSPITAL   

                      address      city state   zip5  beds  phone_number  \
0      1108 ROSS CLARK CIRCLE    DOTHAN    AL  36301   420    3347938701   
1  2505 U S HIGHWAY 431 NORTH      BOAZ    AL  35957   240    2565938310   
2         1701 VETERANS DRIVE  FLORENCE    AL  35630   338    2567688400   
3               702 N MAIN ST       OPP    AL  36467    99    3344933541   
4         101 HOSPITAL CIRCLE   LUVERNE    AL  36049    65    3343353374   

                                        homepage_url  \
0  https://www.southeasthealth.org/southeast-heal...   
1                   https://www.ma

In [ ]:
print(f'# Hospitals: {hospitals_df.shape[0]} \n# Priced objects: {prices_df.shape[0]} \nAverage # priced objects per hospital: {prices_df.shape[0]/hospitals_df.shape[0]}')

#5992 hospitals
#296210747 priced items
#avg of 49434.370327102806 priced items per hospital

# Hospitals: 5992 
# Priced objects: 296210747 
Average # priced objects per hospital: 49434.370327102806


In [ ]:
prices_df['cms_certification_num'] = prices_df['cms_certification_num'].astype('uint32')
prices_df['price'] = prices_df['price'].astype('float32')

#15 sec

In [ ]:
%%time
prices_df = prices_df[prices_df.price >= 0]

#Removes 624 rows, 5 - 7 min

CPU times: user 49.9 s, sys: 2min 53s, total: 3min 43s
Wall time: 6min 49s


In [ ]:
print(prices_df['price'].max())
#16214085.38

16214085.38


In [ ]:
prices_df.dtypes

cms_certification_num     uint32
payer                     object
code                      object
internal_revenue_code     object
units                     object
description               object
inpatient_outpatient      object
price                    float32
code_disambiguator        object
dtype: object

In [ ]:
fi32 = np.finfo(np.float32)
print(fi32.max)
print(fi32.min)
print(prices_df['price'].max())
#16214085.0

3.4028235e+38
-3.4028235e+38
16214085.0


In [ ]:
def priceAvg(cms_id):
    """Returns the average object price for the specific hospital, through cms_certification_num identification"""
    return prices_df['price'].loc[prices_df['cms_certification_num'] == cms_id].mean()

priceAvg(10001)

4601.30859375

In [ ]:
print(hospitals_df.head())

  cms_certification_num                                   name  \
0                010001        SOUTHEAST HEALTH MEDICAL CENTER   
1                010005  MARSHALL MEDICAL CENTERS SOUTH CAMPUS   
2                010006           NORTH ALABAMA MEDICAL CENTER   
3                010007               MIZELL MEMORIAL HOSPITAL   
4                010008            CRENSHAW COMMUNITY HOSPITAL   

                      address      city state   zip5  beds  phone_number  \
0      1108 ROSS CLARK CIRCLE    DOTHAN    AL  36301   420    3347938701   
1  2505 U S HIGHWAY 431 NORTH      BOAZ    AL  35957   240    2565938310   
2         1701 VETERANS DRIVE  FLORENCE    AL  35630   338    2567688400   
3               702 N MAIN ST       OPP    AL  36467    99    3344933541   
4         101 HOSPITAL CIRCLE   LUVERNE    AL  36049    65    3343353374   

                                        homepage_url  \
0  https://www.southeasthealth.org/southeast-heal...   
1                   https://www.ma

In [95]:
%%time
#hospitals_df.assign(avgPrice=0)

for ind in tqdm(range(len(hospitals_df))):
    #print(f"cert num:  {hospitals_df.loc[ind,'cms_certification_num']}")
    hospitals_df.loc[ind, 'avgPrice'] = priceAvg(int(hospitals_df.loc[ind,'cms_certification_num']))
    #print(f"avg price: {hospitals_df.loc[ind, 'avgPrice']}")


ValueError: invalid literal for int() with base 10: '15J200'